Contents of the file `cupy.singularity`:
~~~
Bootstrap: docker

#https://github.com/pytorch/pytorch#docker-image
From: docker://cupy/cupy:latest-python3

%post
    apt-get update
    apt-get install -y graphviz
    apt-get install -y cuda-toolkit-9-2
    pip3 install -U uproot
    pip3 install -U awkward
    pip3 install -U jupyter
    pip3 install -U matplotlib
    pip3 install -U dask
    pip3 install -U distributed
    pip3 install -U graphviz
    pip3 install -U numba

#can be empty
%environment
    JUPYTER_RUNTIME_DIR=$HOME/jupyter-runtime

%runscript
    /bin/bash
~~~

Compile as `sudo singularity build cupy.simg cupy.singularity`.

Run as `JUPYTER_DATA_DIR=/tmp/$USER/jupyter LC_ALL=C singularity exec -B /data -B /nvmedata --nv $SINGULARITY_IMAGE jupyter notebook --certfile $HOME/mycert.pem --keyfile $HOME/mykey.key --ip 0.0.0.0 --port $PORT`.

On `titans.hep.caltech.edu`, the image is in `/nfshome/singularity/images/cupy.simg`.

In [ ]:
#!pip3 install -U pyarrow

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import uproot, cupy, awkward
from awkward.util import numpy as anp
import numpy as np
import dask
import glob
import psutil, os
from collections import OrderedDict
from typing import List, Dict
import math
import numba
import pyarrow

os.environ["NUMBAPRO_NVVM"] = "/usr/local/cuda-9.2/nvvm/lib64/libnvvm.so"
os.environ["NUMBAPRO_LIBDEVICE"] = "/usr/local/cuda-9.2/nvvm/libdevice/"
from numba import cuda

In [ ]:
!nvidia-smi

In [ ]:
awkward.__version__, cupy.__version__, numba.__version__

In [ ]:
infile = '/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/0C2B3A66-B042-E811-8C6D-44A8423DE2C0.root'

In [ ]:
arrays_ev = [
    "PV_npvsGood", "Flag_METFilters", "Flag_goodVertices", "Generator_weight", "genWeight", "HLT_IsoMu24"
]
arrays_jet = [
    "Jet_pt", "Jet_eta", "Jet_phi", "Jet_btagDeepB", "Jet_jetId"
]

arrays_muon = [
    "nMuon", "Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_pfRelIso03_all", "Muon_mediumId", "Muon_charge"
]

arrays_to_load = arrays_jet + arrays_muon

In [ ]:
arrays_gen = [
    "GenPart_pt", "GenPart_eta", "GenPart_phi", "GenPart_mass", "GenPart_pdgId", "GenPart_status"
]

In [ ]:
fi = uproot.open(infile)
tt = fi.get("Events")
arr = tt.array("Jet_pt")

In [ ]:
len(arr)

In [ ]:
arr

In [ ]:
for idx in range(10):
    print("event=", idx, " njet=", len(arr[idx]), " jets_pt=", arr[idx], sep='')

In [ ]:
plt.hist(arr.counts, bins=20);
plt.xlabel("number of jets per event")
plt.ylabel("number of events")

In [ ]:
num_total_dense = len(arr) * np.max(arr.counts)
num_total_dense

In [ ]:
num_total_sparse = len(arr.content)
num_total_sparse

In [ ]:
num_total_sparse / num_total_dense

In [ ]:
muon_charges = tt.array("Muon_charge")

In [ ]:
muon_charges

In [ ]:
muon_charges.content

In [ ]:
muon_charges.offsets

In [ ]:
from collections import Counter
Counter(muon_charges.counts)

In [ ]:
#naive implementation in pure Python
def get_os_muons_slow(muon_charges, out_mask):
    for ic, _c in enumerate(muon_charges):
        
        if _c.shape[0] >= 2:
            
            #find next muon that has an opposite charge, break
            for isecond in range(1, len(_c)):
                if _c[isecond] != _c[0]:
                    
                    #always take the first (leading pt) muon
                    out_mask[ic][0] = True
                    
                    out_mask[ic][isecond] = True
                    break
                
    return

In [ ]:
@numba.jit(nopython=True)
def get_os_muons_numba(muon_charges_content, muon_charges_offsets, muon_mask_out):
    for iev in range(len(muon_charges_offsets) - 1):
        start = muon_charges_offsets[iev]
        end = muon_charges_offsets[iev + 1]
        
        if end-start >= 2:
            ch1 = muon_charges_content[start]
            for imuon in range(start+1, end):
                ch2 = muon_charges_content[imuon]
                if ch2 != ch1:
                    muon_mask_out[start] = True
                    muon_mask_out[imuon] = True
                    break
    return

In [ ]:
@cuda.jit('void(int8[:], int64[:], int8[:])')
def get_os_muons_cuda(muon_charges_content, muon_charges_offsets, muon_mask_out):
    xi = cuda.grid(1)
    xstride = cuda.gridsize(1)
    
    for iev in range(xi, muon_charges_offsets.shape[0]-1, xstride):
        start = muon_charges_offsets[iev]
        end = muon_charges_offsets[iev + 1]
        
        ch1 = muon_charges_content[start]
        
        for imuon in range(start+1, end):
            ch2 = muon_charges_content[imuon]
            if (ch2 != ch1):
                muon_mask_out[start] = 1
                muon_mask_out[imuon] = 1
                break
    return

In [ ]:
def get_os_muons_awkward(muon_charges, out_muon_mask):

    ch = muon_charges
        
    #select events with at least 2 muons
    events_min2_muons = ch.count()>=2
    
    #get the charges of the muons in these events
    ch2 = ch[events_min2_muons]

    #get the index pairs of all muons on an event-by-event basis
    all_muon_pairs = ch2.argcross(ch2)

    #get only those index pairs where the muon is not paired with itself and is paired with another muon with a higher index
    pairs_mask = (all_muon_pairs['0'] != all_muon_pairs['1']) & ((all_muon_pairs['0'] < all_muon_pairs['1']))
    all_muon_pairs = all_muon_pairs[pairs_mask]
    
    #get the pairs with the opposite sign charges
    pairs_with_os = ch2[all_muon_pairs['0']] != ch2[all_muon_pairs['1']]
    
    #get the indices of the pairs with the opposite sign
    idxs = all_muon_pairs[pairs_with_os]

    #get the events that had at least one such good pair
    events_min1_os_pair = idxs['0'].count()>=1
    idxs2 = idxs[events_min1_os_pair]
    bestpair = idxs2[:, 0]
    
    first_muon_idx = bestpair['0']
    second_muon_idx = bestpair['1']

    #set the leading and subleading muons to pass the mask according to the pair
    muon_mask_active = out_muon_mask[events_min2_muons][events_min1_os_pair]
    muon_mask_active.content[muon_mask_active.starts + first_muon_idx] = True
    muon_mask_active.content[muon_mask_active.starts + second_muon_idx] = True
    
    return

In [ ]:
mu_c = cupy.array(muon_charges.content, dtype=np.int8)
mu_o = cupy.array(muon_charges.offsets, dtype=np.int64)

mask_cuda = cupy.zeros_like(mu_c, dtype=np.int8)
mask_numba = np.zeros_like(muon_charges.content, dtype=np.int8)

mask_awkward = awkward.array.jagged.JaggedArray(
    muon_charges.starts,
    muon_charges.stops,
    np.zeros(shape=muon_charges.content.shape, dtype=bool)
)

mask_slow = mask_awkward.copy()

In [ ]:
timing_python = %timeit -o get_os_muons_slow(muon_charges, mask_slow)
timing_python

In [ ]:
timing_cuda = %timeit -n10 -o get_os_muons_cuda[128,1024](mu_c, mu_o, mask_cuda);cuda.synchronize()
timing_cuda

In [ ]:
np.all(np.equal(cupy.asnumpy(mask_cuda), mask_slow.content))

In [ ]:
timing_numba = %timeit -n10 -o get_os_muons_numba(muon_charges.content, muon_charges.offsets, mask_numba)
timing_numba

In [ ]:
np.all(np.equal(cupy.asnumpy(mask_cuda), mask_numba))

In [ ]:
timing_awkward = %timeit -n10 -o get_os_muons_awkward(muon_charges, mask_awkward)

In [ ]:
np.all(np.equal(cupy.asnumpy(mask_cuda), mask_awkward.content))

In [ ]:
avgs = []
stddevs = []
for t in [timing_python, timing_awkward, timing_numba, timing_cuda]:
    avgs += [t.average]
    stddevs += [t.stdev]
    
avgs = np.array(avgs)
stddevs = np.array(stddevs)

normfactor = len(muon_charges)

In [ ]:
plt.bar(range(len(avgs)), normfactor/avgs)
plt.yscale("log")
plt.xticks(range(4), ["python", "awkward", "numba-CPU", "numba-GPU"])
plt.ylabel("Processing rate (Hz)")
plt.title("Events processed: {0:.2E}".format(normfactor))

In [ ]:
muon_mask_opposite_sign = awkward.array.jagged.JaggedArray(
    muon_charges.starts,
    muon_charges.stops,
    cupy.asnumpy(mask_cuda).view(dtype=bool)
)

In [ ]:
#check if we selected only opposite sign muons
selected_charges = muon_charges[muon_mask_opposite_sign]

In [ ]:
selected_charges

In [ ]:
arrs = tt.arrays([b"Muon_pt", b"Muon_eta", b"Muon_phi", b"Muon_tightId", b"Muon_charge"])

In [ ]:
class JaggedStruct(object):
    def __init__(self, offsets, attrs_data, numpy_lib=np):
        self.numpy_lib = numpy_lib
        
        self.offsets = offsets
        self.attrs_data = attrs_data
        
        num_items = None
        for (k, v) in self.attrs_data.items():
            num_items_next = len(v)
            if num_items and num_items != num_items_next:
                raise AttributeError("Mismatched attribute {0}".format(k))
            else:
                num_items = num_items_next
            setattr(self, k, v)
    
        self.mask = self.numpy_lib.ones(num_items, dtype=self.numpy_lib.int8)
        
    @staticmethod
    def from_arraydict(arraydict, prefix, numpy_lib=np):
        ks = [k for k in arraydict.keys() if prefix in str(k, 'ascii')]
        k0 = ks[0]
        return JaggedStruct(
            numpy_lib.array(arraydict[k0].offsets),
            {str(k, 'ascii').replace(prefix, ""): numpy_lib.array(v.content)
             for (k,v) in arraydict.items()},
            numpy_lib=numpy_lib
        )

In [ ]:
@cuda.jit('void(int32[:], int64[:], int8[:])')
def select_opposite_sign_muons_cudakernel(muon_charges_content, muon_charges_offsets, muon_mask_out):
    xi = cuda.grid(1)
    xstride = cuda.gridsize(1)
    
    for iev in range(xi, muon_charges_offsets.shape[0]-1, xstride):
        start = muon_charges_offsets[iev]
        end = muon_charges_offsets[iev + 1]
        
        ch1 = muon_charges_content[start]
        
        for imuon in range(start+1, end):
            ch2 = muon_charges_content[imuon]
            if (ch2 != ch1):
                muon_mask_out[start] = 0
                muon_mask_out[imuon] = 0
                break
    return


@cuda.jit('void(int8[:], int64[:], int8[:])')
def sum_in_event(content, offsets, out):
    xi = cuda.grid(1)
    xstride = cuda.gridsize(1)

    for iev in range(xi, offsets.shape[0]-1, xstride):
        start = offsets[iev]
        end = offsets[iev + 1]
        for ielem in range(start, end):
            out[iev] += content[ielem]

def select_muons_opposite_sign(muons, mask_name=None):
    if not mask_name:
        select_opposite_sign_muons_cudakernel[32,1024](muons.charge, muons.offsets, muons.mask)
    else:
        mask = muons.numpy_lib.ones_like(muons.mask)
        select_opposite_sign_muons_cudakernel[32,1024](muons.charge, muons.offsets, mask)
        setattr(muons, mask_name, mask)
        
def select_muons_pt(muons, ptcut, mask_name=None):
    if not mask_name:
        muons.mask = muons.mask & (muons.pt > ptcut)
    else:
        setattr(muons, mask_name, muons.pt > ptcut)

In [ ]:
muons = JaggedStruct.from_arraydict(arrs, "Muon_", cupy)
select_muons_pt(muons, 30, "mask_subleading")
select_muons_pt(muons, 30, "mask_leading")

In [ ]:
muons.mask_subleading
muons.mask_leading

In [ ]:
sum_out = cupy.zeros_like(muons.offsets, dtype=cupy.int8)
%timeit sum_in_event[128, 1024](muons.mask_subleading, muons.offsets, sum_out);numba.cuda.synchronize()

In [ ]:
awkward.JaggedArray.fromoffsets?

In [ ]:
mask_sl =awkward.JaggedArray.fromoffsets(
    cupy.asnumpy(muons.offsets),
    cupy.asnumpy(muons.mask_subleading),
)

In [ ]:
%timeit mask_sl.sum()